<a href="https://colab.research.google.com/github/meshun074/VBNMF/blob/main/VBNMF.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
import pandas as pd
import torch
import torch.nn.functional as F

In [4]:
from google.colab import drive
drive.mount('/content/drive')


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [5]:
# Path to the CSV file in your Google Drive
filename = '/content/drive/My Drive/Data/sampleclass.csv'

# Read the CSV file into a DataFrame
df = pd.read_csv(filename)

# Display the first few rows of the DataFrame
df.head()


,barcode,sample_type
0,TCGA-D8-A146-01A-31R-A115-07,Primary Tumor
1,TCGA-AQ-A0Y5-01A-11R-A14M-07,Primary Tumor
2,TCGA-C8-A274-01A-11R-A16F-07,Primary Tumor
3,TCGA-BH-A0BD-01A-11R-A034-07,Primary Tumor
4,TCGA-B6-A1KC-01B-11R-A157-07,Primary Tumor


In [6]:

# Path to the CSV file in your Google Drive
file_path = '/content/drive/My Drive/Data/data.csv'

# Read the CSV file into a DataFrame
df1 = pd.read_csv(file_path)




In [7]:
barcode = df['barcode'].values.tolist()

In [8]:
df1.shape

(14629, 1232)

In [9]:
df1.head()

,Unnamed: 0,TCGA-AC-A6IX-06A-11R-A32P-07,TCGA-BH-A18V-06A-11R-A213-07,TCGA-BH-A1ES-06A-12R-A24H-07,TCGA-BH-A1FE-06A-11R-A213-07,TCGA-E2-A15A-06A-11R-A12D-07,TCGA-E2-A15E-06A-11R-A12D-07,TCGA-E2-A15K-06A-11R-A12P-07,TCGA-3C-AAAU-01A-11R-A41B-07,TCGA-3C-AALI-01A-11R-A41B-07,...,TCGA-E9-A1NG-11A-52R-A14M-07,TCGA-E9-A1R7-11A-42R-A14M-07,TCGA-E9-A1RB-11A-33R-A157-07,TCGA-E9-A1RC-11A-33R-A157-07,TCGA-E9-A1RD-11A-33R-A157-07,TCGA-E9-A1RF-11A-32R-A157-07,TCGA-E9-A1RH-11A-34R-A169-07,TCGA-E9-A1RI-11A-41R-A169-07,TCGA-GI-A2C8-11A-22R-A16F-07,TCGA-GI-A2C9-11A-22R-A21T-07
0,ENSG00000000003,6568,1160,4187,4964,3842,946,69,651,418,...,3418,3827,5055,5225,5385,4527,3573,7587,420,4816
1,ENSG00000000005,40,17,1,104,1,0,0,2,2,...,1291,2918,908,2026,1877,1879,770,3528,16,308
2,ENSG00000000419,2448,2809,2148,1808,4899,2356,1778,1872,1818,...,1554,1656,2017,1811,1988,1557,1607,1991,1664,2279
3,ENSG00000000457,3576,1619,1970,575,2361,2380,2546,951,2437,...,909,978,1432,1048,1285,1160,1040,1091,654,1086
4,ENSG00000000460,818,883,555,160,1291,636,866,339,490,...,272,269,378,290,322,275,215,250,84,294


In [10]:
features = df1['Unnamed: 0'].values.tolist()

In [11]:
features[:5]

['ENSG00000000003',
 'ENSG00000000005',
 'ENSG00000000419',
 'ENSG00000000457',
 'ENSG00000000460']

In [12]:
df1.drop(columns=['Unnamed: 0'], inplace=True)

In [13]:
df1.head()

,TCGA-AC-A6IX-06A-11R-A32P-07,TCGA-BH-A18V-06A-11R-A213-07,TCGA-BH-A1ES-06A-12R-A24H-07,TCGA-BH-A1FE-06A-11R-A213-07,TCGA-E2-A15A-06A-11R-A12D-07,TCGA-E2-A15E-06A-11R-A12D-07,TCGA-E2-A15K-06A-11R-A12P-07,TCGA-3C-AAAU-01A-11R-A41B-07,TCGA-3C-AALI-01A-11R-A41B-07,TCGA-3C-AALJ-01A-31R-A41B-07,...,TCGA-E9-A1NG-11A-52R-A14M-07,TCGA-E9-A1R7-11A-42R-A14M-07,TCGA-E9-A1RB-11A-33R-A157-07,TCGA-E9-A1RC-11A-33R-A157-07,TCGA-E9-A1RD-11A-33R-A157-07,TCGA-E9-A1RF-11A-32R-A157-07,TCGA-E9-A1RH-11A-34R-A169-07,TCGA-E9-A1RI-11A-41R-A169-07,TCGA-GI-A2C8-11A-22R-A16F-07,TCGA-GI-A2C9-11A-22R-A21T-07
0,6568,1160,4187,4964,3842,946,69,651,418,1303,...,3418,3827,5055,5225,5385,4527,3573,7587,420,4816
1,40,17,1,104,1,0,0,2,2,52,...,1291,2918,908,2026,1877,1879,770,3528,16,308
2,2448,2809,2148,1808,4899,2356,1778,1872,1818,1281,...,1554,1656,2017,1811,1988,1557,1607,1991,1664,2279
3,3576,1619,1970,575,2361,2380,2546,951,2437,380,...,909,978,1432,1048,1285,1160,1040,1091,654,1086
4,818,883,555,160,1291,636,866,339,490,239,...,272,269,378,290,322,275,215,250,84,294


In [14]:
samples_name_list = df1.columns.tolist()

In [15]:
X=df1.values


In [16]:
# Selecting a guessed
h_len = X.shape[1]
w_len = X.shape[0]
r=615

In [17]:
X = torch.tensor(df1.values, dtype=torch.float32)

In [18]:
import numpy as np
import scipy.special as sc

In [19]:
class vbnmf:
    def __init__(self, X=None, hyperparameters=None, parameters=None):
        #initialize
        self.X=X # data
        self.hyperparameters=hyperparameters
        self.parameters=parameters # names of samples
        W_n=self.X.shape[0] # number of features
        H_n=self.X.shape[1] # number of samples
        self.a_w=hyperparameters[0]
        self.b_w=hyperparameters[1]
        self.a_h=hyperparameters[2]
        self.b_h=hyperparameters[3]
        r = b_w.shape[1]
        self.epoch = parameters[0]
        self.period = parameters[1]
        self.W_1 = np.random.gamma(a_w,np.divide(b_w,a_w))
        self.H_1 = np.random.gamma(a_h,np.divide(b_h,a_h))

    def initializeModel(self):
      # Convert Nan values to 0
      M = ~np.isnan(X)
      x = torch.tensor(np.zeros_like(X), dtype=torch.float32)
      x[M] = X[M]
      self.X=x
      L_w = self.W_1
      L_h = self.H_1
      E_w = self.W_1
      E_h = self.H_1
      Sig_w = self.W_1
      Sig_h = self.H_1

      B = np.zeros(self.epoch)

      gammalnX = sc.gammaln(X + 1)

      for i in range(1, self.epoch + 1):

        LtLv = np.dot(L_w, L_h)
        div = X / LtLv
        Sig_w = L_w * np.dot(div, L_h.T)
        Sig_v = L_h * np.dot(L_w.T, div)

        alpha_w = a_w + Sig_w
        beta_w = 1 / (a_w / b_w + np.dot(M, E_h.T))
        E_w = alpha_w * beta_w

        alpha_h = a_h + Sig_h
        beta_h = 1 / (a_h / b_h + np.dot(E_w.T, M))
        E_h = alpha_h * beta_h

        # Compute lower bound
        if i % self.period == 1 or i == self.epoch:
          self.E_w = E_w;
          self.logE_w = np.log(L_w);
          self.E_h = E_h;
          self.logE_h = np.log(L_h);
          # self.Bound = -np.sum(np.sum(M * np.dot(self.E_w, self.E_h)) + gammalnX) #\
                    # + np.sum(np.sum(-X * ((np.dot(L_w ,self.logE_w) * L_h + L_w * np.dot(L_h ,self.logE_h)) / LtLv - np.log(LtLv)))) \
                    # + np.sum(np.sum(-a_w / b_w * self.E_w - sc.gammaln(a_w) + a_w * np.log(a_w / b_w))) \
                    # + np.sum(np.sum(-a_h / b_w * self.E_h - sc.gammaln(a_h) + a_h * np.log(a_h / b_h))) \
                    # + np.sum(np.sum(sc.gammaln(alpha_w) + alpha_w * (np.log(beta_w) + 1))) \
                    # + np.sum(np.sum(sc.gammaln(alpha_h) + alpha_h * (np.log(beta_h) + 1)))

          # print(f'\nBound = {self.Bound}\t a_ve = {a_h[0]} \t b_ve = {b_h[0]} \t a_tm = {a_w[0]} \t b_tm = {b_w[0]}')


In [20]:
# prior parameters
a_w = np.ones((w_len, r))
b_w = np.ones((w_len, r))
a_h = np.ones((r, h_len))
b_h = np.ones((r, h_len))
hyperparameters = [a_w, b_w, a_h, b_h]
parameters = [20,5]

In [21]:
vb = vbnmf(X,hyperparameters,parameters)
vb.initializeModel()

<ipython-input-19-a9dd9190d2d4>:23: UserWarning: indexing with dtype torch.uint8 is now deprecated, please use a dtype torch.bool instead. (Triggered internally at ../aten/src/ATen/native/IndexingUtils.h:27.)
  x[M] = X[M]


TypeError: sum() received an invalid combination of arguments - got (out=NoneType, axis=NoneType, ), but expected one of:
 * (*, torch.dtype dtype)
      didn't match because some of the keywords were incorrect: out, axis
 * (tuple of ints dim, bool keepdim, *, torch.dtype dtype)
 * (tuple of names dim, bool keepdim, *, torch.dtype dtype)
